In [ ]:
!pip install sounddevice numpy requests -q

In [ ]:
"""
Real-time demo client for the Alquimia TTS runtime
--------------------------------------------------
• POSTs an isolated request and obtains a stream-id
• Streams /stream/<id> (plain JSON lines)
• Maintains a HIGH / LOW jitter-buffer so PortAudio never underruns
• Plays the audio with sounddevice and logs each chunk
"""
import json
import logging
import uuid
from time import perf_counter

import numpy as np
import requests
import sounddevice as sd

logging.basicConfig(level=logging.INFO, format="▶ %(message)s")

ALQUIMIA_SECRET = "alquimiaai-secret"
ALQUIMIA_URL    = "https://runtime.apps.alquimiaai.hostmydemo.online"
EXPERT          = "orpheus"

# jitter-buffer parameters (ms)
HIGH_MS = 3000          # start playback when buffer ≥ HIGH_MS
LOW_MS  = 150          # never drain buffer below LOW_MS

CHUNK_LOG_EVERY = 1    # print every chunk

def invoke(
    query: str,
    speaker_id: str = "tara",
    session_id: str | uuid.UUID = uuid.uuid4(),
) -> None:
    logging.info("📝 Prompt: %s", query)

    # ── 1) isolated inference -> stream_id ────────────────────────────────
    resp = requests.post(
        f"{ALQUIMIA_URL}/infer/chat/isolated/{EXPERT}",
        json={
            "query":       query,
            "extra_data":  {"speaker_id": speaker_id},
            "session_id":  str(session_id),
        },
        headers={"Authorization": f"Bearer {ALQUIMIA_SECRET}"},
        timeout=30,
    )
    resp.raise_for_status()
    stream_id = resp.json()["data"].get("stream_id")
    if not stream_id:
        logging.error("No stream_id returned.")
        return
    logging.info("🆔 stream_id: %s", stream_id)

    # ── 2) open chunked JSON stream ───────────────────────────────────────
    stream_url = f"{ALQUIMIA_URL}/stream/{stream_id}"
    logging.info("📡 Connecting to stream …  %s", stream_url)

    buf            = bytearray()          # raw int16 bytes
    sr             = None
    sd_stream      = None
    BPS            = 2                    # bytes per int16 sample
    chunk_idx      = 0
    t0             = perf_counter()

    with requests.get(
        stream_url,
        headers={"Authorization": f"Bearer {ALQUIMIA_SECRET}"},
        stream=True,
    ) as sresp:
        sresp.raise_for_status()
        for line in sresp.iter_lines():
            if line:
                    decoded_line = line.decode("utf-8")
                    if decoded_line.startswith("data:"):
                        json_line = decoded_line[len("data:"):].strip()
                        try:
                            payload = json.loads(json_line)
                        except Exception:
                            continue

            answer       = payload["answer"]["outputs"]
            is_complete  = payload.get("is_complete", False)

            # append chunk
            chunk = np.array(answer["audio"], dtype=np.int16)
            buf.extend(chunk.tobytes())
            if sr is None:
                sr = int(answer["sampling_rate"])

            # open stream when buffer ≥ HIGH_MS
            if (
                sd_stream is None
                and sr is not None
                and len(buf) >= sr * BPS * HIGH_MS // 1000
            ):
                sd_stream = sd.OutputStream(
                    samplerate=sr,
                    channels=1,
                    dtype="int16",
                    latency="low",
                    blocksize=0,
                )
                sd_stream.start()
                logging.info("🔈 Playback started (buffered %d ms)", HIGH_MS)

            # write while buffer ≥ LOW_MS + one block
            if sd_stream is not None:
                block_bytes = (sd_stream.blocksize or 1024) * BPS
                min_buf     = sr * BPS * LOW_MS // 1000
                while len(buf) - block_bytes >= min_buf:
                    frame = np.frombuffer(buf[:block_bytes], dtype=np.int16).reshape(-1, 1)
                    sd_stream.write(frame)
                    del buf[:block_bytes]

            # logging
            if chunk_idx % CHUNK_LOG_EVERY == 0:
                logging.info(
                    "🔊 chunk %03d | %5d samples | %5.2f s",
                    chunk_idx, chunk.size, perf_counter() - t0
                )
            chunk_idx += 1

            if is_complete:
                logging.info("🏁 Final chunk received — draining buffer")
                break

        # drain remainder
        if sd_stream is not None and buf:
            sd_stream.write(np.frombuffer(buf, dtype=np.int16).reshape(-1, 1))
            sd_stream.stop(); sd_stream.close()

    logging.info("✅ Finished (%d chunks played)", chunk_idx)


In [ ]:
invoke("Hello world! I'm playing this from alquimia runtime")